# Large Scale Production Cost Modeling with [PowerSimulations.jl](https://github.com/NREL-SIIP/PowerSimulations.jl)

**Originally Contributed by**: Clayton Barrows

In [9]:
] up

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 %58.3 %  Updating git-repo `https://github.com/NREL-SIIP/PowerSimulations.jl.git`
Fetching: [========================================>]  100.0 %.0 %  Updating git-repo `https://github.com/NREL-SIIP/PowerGraphics.jl.git`
 Resolving package versions...
 Installed FFMPEG_jll ────── v4.1.0+3
 Installed RecipesPipeline ─ v0.1.7
  Updating `~/Documents/repos/SIIPExamples.jl/Project.toml`
 [no changes]
  Updating `~/Documents/repos/SIIPExamples.jl/Manifest.toml`
  [b22a6f82] ↑ FFMPEG_jll v4.1.0+2 ⇒ v4.1.0+3
  [01d81517] ↑ RecipesPipeline v0.1.6 ⇒ v0.1.7


In [1]:
include("../../script/PowerSystems_examples/US_system.jl")

┌ Info: downloading data...
└ @ Main /Users/cbarrows/Documents/repos/SIIPExamples.jl/script/PowerSystems_examples/US_system.jl:31
┌ Info: formatting data ...
└ @ Main /Users/cbarrows/Documents/repos/SIIPExamples.jl/script/PowerSystems_examples/US_system.jl:58
┌ Info: filtering data to include Texas ...
└ @ Main /Users/cbarrows/Documents/repos/SIIPExamples.jl/script/PowerSystems_examples/US_system.jl:59
┌ Info: formatting wind.csv ...
└ @ Main /Users/cbarrows/Documents/repos/SIIPExamples.jl/script/PowerSystems_examples/US_system.jl:130
┌ Info: formatting solar.csv ...
└ @ Main /Users/cbarrows/Documents/repos/SIIPExamples.jl/script/PowerSystems_examples/US_system.jl:130
┌ Info: formatting hydro.csv ...
└ @ Main /Users/cbarrows/Documents/repos/SIIPExamples.jl/script/PowerSystems_examples/US_system.jl:130
┌ Info: formatting demand.csv ...
└ @ Main /Users/cbarrows/Documents/repos/SIIPExamples.jl/script/PowerSystems_examples/US_system.jl:130
┌ Info: parsing csv files...
└ @ Main /Users/cbarr

## Introduction

This example shows a basic PCM simulation using the system data assembled in the
[US-System example](../../notebook/PowerSystems_examples/US_system.ipynb).

### Dependencies

In [2]:
using SIIPExamples
using PowerSystems
using PowerSimulations
using PowerGraphics
using Logging
pkgpath = dirname(dirname(pathof(SIIPExamples)))
PSY = PowerSystems
PSI = PowerSimulations
plotlyjs()

┌ Info: Precompiling PowerGraphics [5f7eddb3-86b1-49e8-a95a-ddc0f45eea41]
└ @ Base loading.jl:1273
[ Info: PowerGrahics.jl loads Plots.jl. Precompile might take a while


HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-2794430511135231170\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-9397050400986468522\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-2794430511135231170\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

Plots.PlotlyJSBackend()

### Optimization packages
You can use the cbc solver as in one of the other PowerSimulations.jl examples, but on
large problems it's useful to have a solver with better performance. I'll use the Xpress
solver, since I have a license, but others such as Gurobi or CPLEX work well too.

In [3]:
using Xpress
solver = optimizer_with_attributes(Xpress.Optimizer, "MIPRELSTOP" => 0.1, "OUTPUTLOG" => 1)

┌ Info: Xpress: Found license file /Users/cbarrows/Applications/fico-xpress/xpressmp/bin/xpauth.xpr
└ @ Xpress /Users/cbarrows/.julia/packages/Xpress/kcAft/src/license.jl:42
┌ Info: Xpress: Development license detected.
└ @ Xpress /Users/cbarrows/.julia/packages/Xpress/kcAft/src/license.jl:84


MathOptInterface.OptimizerWithAttributes(Xpress.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute,Any}[MathOptInterface.RawParameter("MIPRELSTOP") => 0.1, MathOptInterface.RawParameter("OUTPUTLOG") => 1])

### Load the US `System`.
If you have run the
[US-System example](../../notebook/PowerSystems_examples/US-System.ipynb), the data will
be serialized in the json and H5 format, so we can efficiently deserialize it:

In [4]:
sys = System(joinpath(pkgpath,"US-System", "SIIP", "sys.json"))

┌ Info: Loaded time series from storage file existing=time_series_storage.h5 new=/var/folders/27/2jr8c7gn4j72fvrg4qt81zrw8w_711/T/jl_yjUJmu.h5
└ @ InfrastructureSystems /Users/cbarrows/.julia/packages/InfrastructureSystems/yly1C/src/hdf5_time_series_storage.jl:81


,ConcreteType,SuperTypes,Count
,String,String,Int64
1,Area,AggregationTopology <: Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,8
2,Bus,Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,2000
3,HydroDispatch,HydroGen <: Generator <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,22
4,Line,ACBranch <: Branch <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,2345
5,LoadZone,AggregationTopology <: Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,1
6,PowerLoad,StaticLoad <: ElectricLoad <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,1125
7,RenewableDispatch,RenewableGen <: Generator <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,109
8,ThermalStandard,ThermalGen <: Generator <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,417
9,Transformer2W,ACBranch <: Branch <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,861


### Selecting flow limited lines
Since PowerSimulations will apply constraints by component type (e.g. Line), we need to
change the component type of the lines on which we want to enforce flow limits. So, let's
change the device type of certain branches from Line to MonitoredLine differentiate
treatment when we build the model. Here, we can select inter-regional lines, or lines
above a voltage threshold.

In [5]:
for line in get_components(Line, sys)
    if (get_basevoltage(get_from(get_arc(line))) >= 230.0) &&
       (get_basevoltage(get_to(get_arc(line))) >= 230.0)
        #if get_area(get_from(get_arc(line))) != get_area(get_to(get_arc(line)))
        @info "Changing $(get_name(line)) to MonitoredLine"
        convert_component!(MonitoredLine, line, sys)
    end
end

┌ Info: Changing branch103799 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch101463 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch103577 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch101301 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch102417 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch103998 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch101303 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch103086 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch101307 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch104069 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch103873 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch103365 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch101866 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch101896 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch102045 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch102604 to Monitor

┌ Info: Changing branch103427 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch101075 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch101768 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch101174 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch101782 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch101500 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch101645 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch101196 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch101513 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch102076 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch103311 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch103593 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch101855 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch101673 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch103398 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch101575 to Monitor

┌ Info: Changing branch103508 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch101940 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch102296 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch101195 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch102366 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch101074 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch102364 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch103100 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch101550 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch102921 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch101202 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch101033 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch101046 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch103015 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch103181 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing branch103750 to Monitor

### Create a `template`
Now we can create a `template` that applies an unbounded formulation to `Line`s and the standard
flow limited formulation to `MonitoredLine`s.

In [6]:
branches = Dict{Symbol, DeviceModel}(
    :L => DeviceModel(Line, StaticLineUnbounded),
    :T => DeviceModel(Transformer2W, StaticTransformer),
    :TT => DeviceModel(TapTransformer, StaticTransformer),
    :ML => DeviceModel(MonitoredLine, StaticLine),
    :DC => DeviceModel(HVDCLine, HVDCDispatch),
)

devices = Dict(
    :Generators => DeviceModel(ThermalStandard, ThermalStandardUnitCommitment),
    :Ren => DeviceModel(RenewableDispatch, RenewableFullDispatch),
    :Loads => DeviceModel(PowerLoad, StaticPowerLoad),
    :HydroROR => DeviceModel(HydroDispatch, FixedOutput),
    :RenFx => DeviceModel(RenewableFix, FixedOutput),
    :ILoads => DeviceModel(InterruptibleLoad, InterruptiblePowerLoad),
)

template = OperationsProblemTemplate(DCPPowerModel, devices, branches, Dict());

### Build and execute single step problem

In [7]:
op_problem =
    OperationsProblem(GenericOpProblem, template, sys; optimizer = solver, horizon = 24, slack_variables = false, use_parameters = true)

res =solve!(op_problem)

 
Reading Problem 
Problem Statistics
           0 (      0 spare) rows
           0 (      0 spare) structural columns
           0 (      0 spare) non-zero elements
Global Statistics
           0 entities        0 sets        0 set members


┌ Warning: The data doesn't include devices of type InterruptibleLoad, consider changing the device models
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/9nQ1Q/src/devices_models/device_constructors/common/constructor_validations.jl:3
┌ Warning: Data doesn't contain generators with ramp limits, consider adjusting your formulation
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/9nQ1Q/src/devices_models/devices/thermal_generation.jl:385
┌ Warning: The data doesn't include devices of type RenewableFix, consider changing the device models
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/9nQ1Q/src/devices_models/device_constructors/common/constructor_validations.jl:3
┌ Warning: The data doesn't include devices of type TapTransformer, consider changing the device models
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/9nQ1Q/src/devices_models/device_constructors/common/constructor_validations.jl:3
┌ Warnin

 
Reading Problem 
Problem Statistics
           0 (      0 spare) rows
           0 (      0 spare) structural columns
           0 (      0 spare) non-zero elements
Global Statistics
           0 entities        0 sets        0 set members
FICO Xpress v8.8.0, Hyper, solve started 14:15:41, May 7, 2020
Heap usage: 108MB (peak 109MB, 1MB system)
Minimizing MILP 
Original problem has:
    479592 rows       197616 cols      1097819 elements     30024 globals
Presolved problem has:
     83815 rows        90711 cols       424118 elements     18963 globals
LP relaxation tightened
Presolve finished in 3 seconds
Heap usage: 164MB (peak 392MB, 1MB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 3.90e-03,  1.43e+03] / [ 9.77e-04,  2.00e+00]
  RHS and bounds [min,max] : [ 1.51e-04,  1.00e+06] / [ 5.42e-20,  4.35e+01]
  Objective      [min,max] : [ 7.21e-01,  1.22e+03] / [ 7.21e-01,  1.99e+04]

Symmetric problem: generators: 206

,Time,gen13011,gen13132,gen13103,gen13114,gen13169,gen13345
,DateTime,Float64,Float64,Float64,Float64,Float64,Float64
1,2016-01-01T00:00:00,0.0,0.0,0.0,1.0,1.0,1.0
2,2016-01-01T01:00:00,0.0,0.0,0.0,1.0,1.0,1.0
3,2016-01-01T02:00:00,0.0,0.0,0.0,1.0,1.0,1.0
4,2016-01-01T03:00:00,0.0,0.0,0.0,1.0,1.0,1.0
5,2016-01-01T04:00:00,0.0,0.0,0.0,1.0,1.0,1.0
6,2016-01-01T05:00:00,0.0,0.0,0.0,1.0,1.0,1.0
7,2016-01-01T06:00:00,0.0,0.0,0.0,1.0,1.0,1.0
8,2016-01-01T07:00:00,0.0,0.0,0.0,1.0,1.0,1.0
9,2016-01-01T08:00:00,0.0,0.0,0.0,1.0,1.0,1.0


### Analyze results

In [19]:
res.parameter_values[:P_HydroDispatch]

,gen13463,gen13461,gen13159,gen12997,gen12977,gen13182,gen13201,gen13462
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0438633,0.158781,0.138807,0.100851,0.0945973,0.193246,0.0,0.150445
2,0.0381989,0.138276,0.120882,0.0878276,0.0823811,0.168291,0.0,0.131017
3,0.0190268,0.0688753,0.0602111,0.0437468,0.0410339,0.0838253,0.0,0.0652594
4,0.00852091,0.0308449,0.0269648,0.0195915,0.0183765,0.0375401,0.0,0.0292256
5,0.00692324,0.0250615,0.0219089,0.0159181,0.0149309,0.0305013,0.0,0.0237458
6,0.00697165,0.0252368,0.0220621,0.0160294,0.0150353,0.0307146,0.0,0.0239118
7,0.0294503,0.106607,0.0931965,0.0677126,0.0635135,0.129747,0.0,0.10101
8,0.0299272,0.108334,0.094706,0.0688094,0.0645422,0.131849,0.0,0.102646
9,0.0284107,0.102844,0.0899067,0.0653224,0.0612715,0.125167,0.0,0.0974447


In [29]:
occursin("Load", String(:P_PowerLoad))

true

In [8]:
fuel_plot(res, sys, load = true)

data: [
  "scatter with fields fill, fillcolor, line, mode, name, stackgroup, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, stackgroup, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, stackgroup, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, stackgroup, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, stackgroup, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, stackgroup, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y"
]

layout: "layout with fields margin, title, and yaxis"

data: [
  "bar with fields barmode, marker, name, stackgroup, type, x, and y",
  "bar with fields barmode, marker, name, stackgroup, type, x, and y",
  "bar with fields barmode, marker, name, stackgroup, type, x, and y",
  "bar with fields barmode, marker, name, stackgroup, type, x, and y",
  "bar with fields barmode, marker, name, stackgroup, type, x, and y",
  "bar with fields barmode, marker, name, stackgroup, type, x, and y"
]

layout: "layout with fields barmode, color, margin, title, and yaxis"

## Sequential Simulation
In addition to defining the formulation template, sequential simulations require
definitions for how information flows between problems.

In [9]:
sim_folder = mkpath(joinpath(pkgpath, "Texas-sim"), )
stages_definition = Dict(
    "UC" => Stage(GenericOpProblem, template, sys, solver; slack_variables = true)
)
order = Dict(1 => "UC")
horizons = Dict("UC" => 24)
intervals = Dict("UC" => (Hour(24), Consecutive()))
cache = Dict(("UC",) => TimeStatusChange(PSY.ThermalStandard, PSI.ON))
DA_sequence = SimulationSequence(
    step_resolution = Hour(24),
    order = order,
    horizons = horizons,
    intervals = intervals,
    ini_cond_chronology = InterStageChronology(),
    cache = cache, #needed for ThermalStandardUC not for Basic
)

┌ Warning: No Feedforward Chronologies have been defined. This configuration assummes that there is no information passing between stages
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/9nQ1Q/src/core/simulation_sequence.jl:127
┌ Warning: Single stage detected, the default Initial Condition Chronology is IntraStageChronology(), other values will be ignored.
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/9nQ1Q/src/core/simulation_sequence.jl:131


Feed Forward Chronology
-----------------------

Initial Condition Chronology
----------------------------

1



### Define and build a simulation

In [12]:
mkpath("Texas-sim")
sim = Simulation(
    name = "Texas-test",
    steps = 10,
    stages = stages_definition,
    stages_sequence = DA_sequence,
    simulation_folder = "Texas-sim"
)
build!(sim, console_level = Logging.Info, file_level = Logging.Debug)

┌ Info: No Feedforward Chronologies defined
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/9nQ1Q/src/core/simulation.jl:287
[ Info: Building Stage 1-UC
┌ Warning: The data doesn't include devices of type InterruptibleLoad, consider changing the device models
└ @ PowerSimulations ~/.julia/packages/PowerSimulations/9nQ1Q/src/devices_models/device_constructors/common/constructor_validations.jl:3


 
Reading Problem 
Problem Statistics
           0 (      0 spare) rows
           0 (      0 spare) structural columns
           0 (      0 spare) non-zero elements
Global Statistics
           0 entities        0 sets        0 set members


┌ Warning: Data doesn't contain generators with ramp limits, consider adjusting your formulation
└ @ PowerSimulations ~/.julia/packages/PowerSimulations/9nQ1Q/src/devices_models/devices/thermal_generation.jl:385
┌ Warning: The data doesn't include devices of type RenewableFix, consider changing the device models
└ @ PowerSimulations ~/.julia/packages/PowerSimulations/9nQ1Q/src/devices_models/device_constructors/common/constructor_validations.jl:3
┌ Warning: The data doesn't include devices of type TapTransformer, consider changing the device models
└ @ PowerSimulations ~/.julia/packages/PowerSimulations/9nQ1Q/src/devices_models/device_constructors/common/constructor_validations.jl:3
┌ Warning: The data doesn't include devices of type HVDCLine, consider changing the device models
└ @ PowerSimulations ~/.julia/packages/PowerSimulations/9nQ1Q/src/devices_models/device_constructors/common/constructor_validations.jl:3
[ Info: Serialized time series data to Texas-sim/Texas-test/1/models_json

### Execute the simulation

In [13]:
sim_results = execute!(sim)

Executing Step 1


[ Info: Starting run step-1-stage-UC 2016-01-01T00:00:00
┌ Info: JuMP.optimize! completed
│   timed_log =
│    Dict{Symbol,Any} with 3 entries:
│      :timed_solve_time  => 27.1315
│      :solve_bytes_alloc => 980846824
└      :sec_in_gc         => 0.455257


 
Reading Problem 
Problem Statistics
           0 (      0 spare) rows
           0 (      0 spare) structural columns
           0 (      0 spare) non-zero elements
Global Statistics
           0 entities        0 sets        0 set members
FICO Xpress v8.8.0, Hyper, solve started 15:50:56, May 7, 2020
Heap usage: 118MB (peak 120MB, 2MB system)
Minimizing MILP 
Original problem has:
    479592 rows       293616 cols      1193819 elements     30024 globals
Presolved problem has:
    112759 rows       215259 cols       596161 elements     18963 globals
LP relaxation tightened
Presolve finished in 3 seconds
Heap usage: 195MB (peak 414MB, 2MB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 3.90e-03,  1.43e+03] / [ 9.77e-04,  2.00e+00]
  RHS and bounds [min,max] : [ 1.51e-04,  1.00e+06] / [ 1.51e-04,  4.35e+01]
  Objective      [min,max] : [ 7.21e-01,  1.00e+06] / [ 7.21e-01,  8.00e+06]

Symmetric problem: generators: 238

[ Info: Starting run step-2-stage-UC 2016-01-02T00:00:00


FICO Xpress v8.8.0, Hyper, solve started 15:51:29, May 7, 2020
Heap usage: 226MB (peak 700MB, 2MB system)
Minimizing MILP 
Original problem has:
    479592 rows       293616 cols      1193819 elements     30024 globals
Presolved problem has:
    113047 rows       215547 cols       594553 elements     18963 globals
LP relaxation tightened
Presolve finished in 3 seconds
Heap usage: 334MB (peak 700MB, 2MB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 3.90e-03,  1.43e+03] / [ 9.77e-04,  2.00e+00]
  RHS and bounds [min,max] : [ 2.78e-06,  1.00e+06] / [ 2.78e-06,  4.35e+01]
  Objective      [min,max] : [ 7.21e-01,  1.00e+06] / [ 7.21e-01,  8.00e+06]

Symmetric problem: generators: 240, support set: 3936
 Number of orbits: 1392, largest orbit: 6, chains: 6
 Row orbits: 3626, row support: 10062
Will try to keep branch and bound tree memory usage below 8.6GB
Starting concurrent solve with dual, primal and barrier (6 threads)

...

┌ Info: JuMP.optimize! completed
│   timed_log =
│    Dict{Symbol,Any} with 3 entries:
│      :timed_solve_time  => 9.47101
│      :solve_bytes_alloc => 23224112
└      :sec_in_gc         => 0.0


[ Info: Starting run step-10-stage-UC 2016-01-10T00:00:00


FICO Xpress v8.8.0, Hyper, solve started 15:53:43, May 7, 2020
Heap usage: 227MB (peak 744MB, 2MB system)
Minimizing MILP 
Original problem has:
    479592 rows       293616 cols      1193819 elements     30024 globals
Presolved problem has:
    113047 rows       215547 cols       594553 elements     18963 globals
LP relaxation tightened
Presolve finished in 3 seconds
Heap usage: 336MB (peak 744MB, 2MB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 3.90e-03,  1.43e+03] / [ 9.77e-04,  2.00e+00]
  RHS and bounds [min,max] : [ 6.54e-05,  1.00e+06] / [ 6.54e-05,  4.35e+01]
  Objective      [min,max] : [ 7.21e-01,  1.00e+06] / [ 7.21e-01,  8.00e+06]

Symmetric problem: generators: 228, support set: 3936
 Number of orbits: 1392, largest orbit: 6, chains: 6
 Row orbits: 3614, row support: 10038
Will try to keep branch and bound tree memory usage below 9.4GB
Starting concurrent solve with dual, primal and barrier (6 threads)

┌ Info: JuMP.optimize! completed
│   timed_log =
│    Dict{Symbol,Any} with 3 entries:
│      :timed_solve_time  => 12.1132
│      :solve_bytes_alloc => 23224112
└      :sec_in_gc         => 0.0
┌ Info: 
│  ──────────────────────────────────────────────────────────────────────────────
│                                        Time                   Allocations      
│                                ──────────────────────   ───────────────────────
│        Tot / % measured:             189s / 100%            15.4GiB / 100%     
│ 
│  Section               ncalls     time   %tot     avg     alloc   %tot      avg
│  ──────────────────────────────────────────────────────────────────────────────
│  Execute Simulation         1     189s   100%    189s   15.4GiB  100%   15.4GiB
│    Execution Step 1         1    34.7s  18.4%   34.7s   2.53GiB  16.5%  2.53GiB
│      Execution Stage 1      1    34.6s  18.3%   34.6s   2.53GiB  16.5%  2.53GiB
│        Run Stage 1          1    32.4s  17.1%   32.4s

Simulation Results Reference

Results Folder: Texas-sim/Texas-test/1/results

Reference Tables

stage-UC

P__ThermalStandard: dataframe size (10, 3)

P__RenewableDispatch: dataframe size (10, 3)

Fp__Transformer2W: dataframe size (10, 3)

parameter_P_PowerLoad: dataframe size (10, 3)

γ⁺__P: dataframe size (10, 3)

Fp__MonitoredLine: dataframe size (10, 3)

parameter_P_RenewableDispatch: dataframe size (10, 3)

Fp__Line: dataframe size (10, 3)

On__ThermalStandard: dataframe size (10, 3)

γ⁻__P: dataframe size (10, 3)

parameter_P_HydroDispatch: dataframe size (10, 3)

Start__ThermalStandard: dataframe size (10, 3)

Stop__ThermalStandard: dataframe size (10, 3)

theta: dataframe size (10, 3)

stage-UC

time length: 24

### Load and analyze results

In [14]:
uc_results = load_simulation_results(sim_results, "UC");

fuel_plot(uc_results, sys, load = true)

data: [
  "scatter with fields fill, fillcolor, line, mode, name, stackgroup, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, stackgroup, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, stackgroup, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, stackgroup, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, stackgroup, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, stackgroup, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, stackgroup, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, stackgroup, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y"
]

layout: "layout with fields margin, title, and yaxis"

data: [
  "bar with fields barmode, marker, name, stackgroup, type, x, and y",
  "bar with fields barmode, marker, name, stackgroup, type, x, and y",
  "bar with fields barmode, marker, name, stackgroup, type, x, and y",
  "bar with fields barmode, marker, name, stackgroup, type, x, and y",
  "bar with fields barmode, marker, name, stackgroup, type, x, and y",
  "bar with fields barmode, marker, name, stackgroup, type, x, and y",
  "bar with fields barmode, marker, name, stackgroup, type, x, and y",
  "bar with fields barmode, marker, name, stackgroup, type, x, and y"
]

layout: "layout with fields barmode, color, margin, title, and yaxis"

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*